In [5]:
# import time
# time.sleep(5000)

In [1]:
competition_name = "deep-learning-spring-2025-project-1"

### Import packages

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import optuna
import numpy as np
import pandas as pd
from datetime import datetime
import os

from data_loader import get_cifar10_dataloaders, get_test_dataloader
from trainer import train_model
from model import ResNet18

/scratch/gk2657/DLSP25-Project1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure the device

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [4]:
# See the total number of trainable parameters
def num_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
model = ResNet18()
num_params(model)

4903242

In [6]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    num_epochs = trial.suggest_int("num_epochs", 10, 50)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    momentum = trial.suggest_uniform("momentum", 0.5, 0.9) if optimizer_name == "SGD" else None

    # Suggest data transformations
    transform = transforms.Compose([
        # add random crop and padding
        transforms.RandomHorizontalFlip(trial.suggest_float("h_flip", 0.0, 1.0)),
        transforms.RandomRotation(trial.suggest_int("rotation", 0, 30)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize with mean and std of CIFAR-10
    ])
    
    train_loader, valid_loader = get_cifar10_dataloaders(
        transform,
        subset_percent=1.0, 
        valid_size=0.1,
        batch_size=batch_size,
        num_workers=4
    )

    # Define model
    model = ResNet18()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model_name = "resnet"

    # Print the current hyperparameters, transformations, and model name
    print("-" * 50)
    print(f"Trial {trial.number}:")
    print(f"Model: {model_name}")
    print(f"Trainable Parameters: {num_params(model)}")
    print(f"Epochs: {num_epochs}")
    print(f"Batch Size: {batch_size}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Optimizer: {optimizer_name}")
    if optimizer_name == "SGD":
        print(f"Momentum: {momentum}")
    print(f"Transformations: {transform}")
    print("- " * 25)

    # Define optimizer
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss()

    # Training
    best_val_accuracy = train_model(
        trial, model, train_loader, criterion, optimizer, 
        valid_loader=valid_loader, num_epochs=num_epochs, device=device)
    
    # Checkpoint the model with the best validation accuracy
    model_filename = f"model_trial_{trial.number}_val_acc_{best_val_accuracy:.4f}.pth"
    study_name = trial.study.study_name
    checkpoint_dir = f"checkpoints_{study_name}"
    model_path = os.path.join(checkpoint_dir, model_filename)
    
    # Create a directory for checkpoints if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Save the model state_dict
    torch.save(model.state_dict(), model_path)
    print(f"Model checkpoint saved to {model_path}")

    return best_val_accuracy

In [7]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
study_name = f"study_{timestamp}"

study = optuna.create_study(direction="maximize", study_name=study_name)
study.optimize(objective, n_trials=20)

print("Best trial:", study.best_trial.number)
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

[I 2025-03-09 12:42:46,559] A new study created in memory with name: study_2025-03-09_12-42-46


--------------------------------------------------
Trial 0:
Model: resnet
Trainable Parameters: <function num_params at 0x14f0a0a4b940>
Epochs: 46
Batch Size: 256
Learning Rate: 0.005965107711939086
Optimizer: RMSprop
Transformations: Compose(
    RandomHorizontalFlip(p=0.06640679932192928)
    RandomRotation(degrees=[-29.0, 29.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/46], Batch [100/176], Loss: 1.7964
  Validation Accuracy after Epoch 1: 0.3694
  Epoch [2/46], Batch [100/176], Loss: 1.5153
  Validation Accuracy after Epoch 2: 0.4332
  Epoch [3/46], Batch [100/176], Loss: 1.2579
  Validation Accuracy after Epoch 3: 0.5672
  Epoch [4/46], Batch [100/176], Loss: 1.0908
  Validation Accuracy after Epoch 4: 0.5522
  Epoch [5/46], Batch [100/176], Loss: 1.0960
  Validation Accuracy after Epoch 5: 0.5778
  Epoch [6/46], Batch [100/17

[I 2025-03-09 12:46:53,604] Trial 0 finished with value: 0.8294 and parameters: {'num_epochs': 46, 'batch_size': 256, 'learning_rate': 0.005965107711939086, 'optimizer': 'RMSprop', 'h_flip': 0.06640679932192928, 'rotation': 29}. Best is trial 0 with value: 0.8294.


Model checkpoint saved to checkpoints_study_2025-03-09_12-42-46/model_trial_0_val_acc_0.8294.pth
--------------------------------------------------
Trial 1:
Model: resnet
Trainable Parameters: <function num_params at 0x14f0a0a4b940>
Epochs: 19
Batch Size: 256
Learning Rate: 0.0003596858499243684
Optimizer: Adam
Transformations: Compose(
    RandomHorizontalFlip(p=0.9912249999453346)
    RandomRotation(degrees=[-18.0, 18.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/19], Batch [100/176], Loss: 1.2905
  Validation Accuracy after Epoch 1: 0.5262
  Epoch [2/19], Batch [100/176], Loss: 0.8953
  Validation Accuracy after Epoch 2: 0.6492
  Epoch [3/19], Batch [100/176], Loss: 0.8088
  Validation Accuracy after Epoch 3: 0.7092
  Epoch [4/19], Batch [100/176], Loss: 0.6357
  Validation Accuracy after Epoch 4: 0.7414
  Epoch [5/19], Batch [10

[I 2025-03-09 12:48:38,917] Trial 1 finished with value: 0.813 and parameters: {'num_epochs': 19, 'batch_size': 256, 'learning_rate': 0.0003596858499243684, 'optimizer': 'Adam', 'h_flip': 0.9912249999453346, 'rotation': 18}. Best is trial 0 with value: 0.8294.


...

  Validation Accuracy after Epoch 2: 0.6432
  Trial pruned due to no improvement.
--------------------------------------------------
Trial 18:
Model: resnet
Trainable Parameters: <function num_params at 0x14f0a0a4b940>
Epochs: 27
Batch Size: 32
Learning Rate: 0.0007696639701485933
Optimizer: Adam
Transformations: Compose(
    RandomHorizontalFlip(p=0.3634967949319905)
    RandomRotation(degrees=[-13.0, 13.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/27], Batch [100/1407], Loss: 1.6089
  Epoch [1/27], Batch [200/1407], Loss: 1.6104
  Epoch [1/27], Batch [300/1407], Loss: 1.4881
  Epoch [1/27], Batch [400/1407], Loss: 1.4963
  Epoch [1/27], Batch [500/1407], Loss: 1.4260
  Epoch [1/27], Batch [600/1407], Loss: 0.9913
  Epoch [1/27], Batch [700/1407], Loss: 1.5128
  Epoch [1/27], Batch [800/1407], Loss: 1.0035
  Epoch [1/27], Batch [9

[I 2025-03-09 13:35:36,402] Trial 18 finished with value: 0.873 and parameters: {'num_epochs': 27, 'batch_size': 32, 'learning_rate': 0.0007696639701485933, 'optimizer': 'Adam', 'h_flip': 0.3634967949319905, 'rotation': 13}. Best is trial 14 with value: 0.8796.


  Validation Accuracy after Epoch 27: 0.8730
Trial 18 complete. Best Validation Accuracy: 0.8730

Model checkpoint saved to checkpoints_study_2025-03-09_12-42-46/model_trial_18_val_acc_0.8730.pth
--------------------------------------------------
Trial 19:
Model: resnet
Trainable Parameters: <function num_params at 0x14f0a0a4b940>
Epochs: 27
Batch Size: 128
Learning Rate: 0.0006359049822704928
Optimizer: Adam
Transformations: Compose(
    RandomHorizontalFlip(p=0.33686194030475763)
    RandomRotation(degrees=[-13.0, 13.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/27], Batch [100/352], Loss: 1.4819
  Epoch [1/27], Batch [200/352], Loss: 1.3383
  Epoch [1/27], Batch [300/352], Loss: 1.0843
  Validation Accuracy after Epoch 1: 0.5462
  Epoch [2/27], Batch [100/352], Loss: 1.0057
  Epoch [2/27], Batch [200/352], Loss: 0.8160
  Epoch [2

[I 2025-03-09 13:38:07,286] Trial 19 finished with value: 0.865 and parameters: {'num_epochs': 27, 'batch_size': 128, 'learning_rate': 0.0006359049822704928, 'optimizer': 'Adam', 'h_flip': 0.33686194030475763, 'rotation': 13}. Best is trial 14 with value: 0.8796.


  Validation Accuracy after Epoch 27: 0.8642
Trial 19 complete. Best Validation Accuracy: 0.8650

Model checkpoint saved to checkpoints_study_2025-03-09_12-42-46/model_trial_19_val_acc_0.8650.pth
Best trial: 14
Best hyperparameters: {'num_epochs': 40, 'batch_size': 32, 'learning_rate': 0.0013132045476528717, 'optimizer': 'Adam', 'h_flip': 0.30078387175615195, 'rotation': 8}
Best validation accuracy: 0.8796


### Load Checkpoint

In [8]:
model = ResNet18()
model.to(device)

# Load the latest checkpoint
checkpoint_dir = f"checkpoints_{study_name}"
checkpoint = torch.load(f"{checkpoint_dir}/model_trial_14_val_acc_0.8796.pth")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize with mean and std of CIFAR-10
])

from trainer import evaluate_model
_, valid_loader = get_cifar10_dataloaders(
    transform,
    subset_percent=1, 
    valid_size=0.1,
    batch_size=128,
    num_workers=4
)

acc = evaluate_model(model, valid_loader, device)
print("Acc:", acc)

Acc: 0.8804


### Run model on test data

In [12]:
# Generate submission file with test data
test_loader = get_test_dataloader()

model.eval()
predictions = []

with torch.no_grad():
    for images, in test_loader:
        images = images.to(device)
        outputs = model(images) 
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({'ID': np.arange(len(predictions)), 'Labels': predictions})
submission.to_csv('submission_88_04.csv', index=False)
print("submission file saved.")

submission file saved.


In [23]:
# import kaggle
# kaggle.api.competition_submit(
#     file_name="submission.csv",
#     message="test",
#     competition=competition_name
# )